# Worksheet

[FORM](https://forms.gle/pQ4Fx4uV7quwasfB8)

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

np.random.seed(123)

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.914369396699439, 5.997345446583585, 5.2829784980519925, 3.493705286081908, 4.4213997480314635, 6.651436537097151, 2.573320756606926, 4.571087371143823, 6.265936258705534, 4.133259597734899]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
np.random.seed(123)

s2 = np.random.normal(8, 1, 10).tolist()
print(s2)

[6.914369396699438, 8.997345446583585, 8.282978498051992, 6.493705286081908, 7.4213997480314635, 9.651436537097151, 5.573320756606925, 7.571087371143823, 9.265936258705533, 7.133259597734899]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
np.random.seed(123)

data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.133259597734899, 6.265936258705534, 4.571087371143823, 2.573320756606926, 6.651436537097151, 7.133259597734899, 4.4213997480314635, 9.265936258705533, 7.571087371143823, 5.573320756606925]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

What are the parameter defines the normal distribution: mean and variance.

P(s1) mean_1 var_1

P(s2) mean_2 var_2

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++', random_state=123).fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)), len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.133259597734899, 4.571087371143823, 2.573320756606926, 4.4213997480314635, 5.573320756606925]
[6.265936258705534, 6.651436537097151, 7.133259597734899, 9.265936258705533, 7.571087371143823]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 4.2544776460248075,  mean_2 = 7.377531204677387
var_1 = 0.9416868317153195,  var_2 = 1.0852126091626229


e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] /prob_x)
    prob_s1_x.append( prob_s[1] * pdf_i[1] /prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


4.133259597734899
Probability of coming from S_1 = 0.9900699208348235
Probability of coming from S_2 = 0.00993007916517656

6.265936258705534
Probability of coming from S_1 = 0.16592069452994254
Probability of coming from S_2 = 0.8340793054700575

4.571087371143823
Probability of coming from S_1 = 0.9686058508217553
Probability of coming from S_2 = 0.03139414917824461

2.573320756606926
Probability of coming from S_1 = 0.9997630297984637
Probability of coming from S_2 = 0.00023697020153632458

6.651436537097151
Probability of coming from S_1 = 0.05346358708543721
Probability of coming from S_2 = 0.9465364129145628

7.133259597734899
Probability of coming from S_1 = 0.010926472404510433
Probability of coming from S_2 = 0.9890735275954896

4.4213997480314635
Probability of coming from S_1 = 0.9788816211608774
Probability of coming from S_2 = 0.02111837883912263

9.265936258705533
Probability of coming from S_1 = 3.7086313542847636e-06
Probability of coming from S_2 = 0.9999962913686458



f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [ ]:
prob_c = [  , sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), ... ]
var = [ ... , ... ]
print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))